In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "tensorflow >2.2.0"

In [ ]:
# duplicates.py

import hashlib
import PIL
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
import numpy as np

def file_hash(filepath):
    with open(filepath, 'rb') as f:
        return md5(f.read()).hexdigest()

import os
os.getcwd()

os.chdir('/content/drive/MyDrive/Kaggle-Autism/consolidated/Non_Autistic')
os.getcwd()

file_list = os.listdir() 
print(len(file_list))

import hashlib, os
duplicates = []
hash_keys = dict()
for index, filename in  enumerate(os.listdir('.')):  #listdir('.') = current directory
    if os.path.isfile(filename):
        with open(filename, 'rb') as f:
            filehash = hashlib.md5(f.read()).hexdigest()
        if filehash not in hash_keys:
            hash_keys[filehash] = index
        else:
            duplicates.append((index,hash_keys[filehash]))

print(duplicates)

for index in duplicates:
    os.remove(file_list[index[0]])

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import time
from shutil import copyfile
import keras
from keras.utils import plot_model
import matplotlib.pyplot as plt

class MLEXPS:

    def __init__(self):
        print('MLEXPS v3')
        self.topic = 'TOPIC'
        self.baseFolder = '/content/drive/MyDrive/Kaggle-Autism'
        self.exprTimeStamp = 'timestamp'
        self.exprFilePath = ''
        self.exprWeightPath = ''
        self.copyFileList = []
        self.currModel = None
        self.currArgs = None
        self.models = []
        self.argList = []
        self.generator = False
        return

    def startExprQ(self):
        if(len(self.models) != len(self.argList)):
            print("Models and Args do not match up.")
            return
        print("Length of queue:", len(self.models))
        for i, expr in enumerate(self.models):
            self.setCurrModel(expr)
            self.setCurrArgs(self.argList[i])
            self.startExpr()
            pass
        return

    def startExpr(self):
        self.currModel.summary()
        self.setupExprDir()
        checkpoint = keras.callbacks.ModelCheckpoint(self.exprWeightPath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        csv = keras.callbacks.CSVLogger(self.exprFilePath + '/logs/training/csvlog.csv', separator=',')
        cb = [checkpoint, csv]
        if 'callbacks' in self.currArgs:
            self.currArgs['callbacks'].append(checkpoint)
            self.currArgs['callbacks'].append(csv)
        else:
            self.currArgs['callbacks'] = [checkpoint, csv]
        if self.generator:
            history = self.currModel.fit_generator(**self.currArgs)
        else:
            history = self.currModel.fit(**self.currArgs)
        self.saveFigures(history)

        self.cleanUpWeights()
        return

    def saveFigures(self, history):

        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.savefig(self.exprFilePath + '/logs/training/accuracy.png')
        plt.close()

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='lower right')
        plt.savefig(self.exprFilePath + '/logs/training/loss.png')
        plt.close()

        plt.rcParams['figure.figsize'] = [10,5]
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.suptitle('Model Stats')
        ax1.set_title("Model Accuracy")
        ax1.set(xlabel='Epoch', ylabel='Accuracy')
        ax1.plot(history.history['accuracy'])
        ax1.plot(history.history['val_accuracy'])
        ax1.legend(['Train', 'Test'], loc='lower right')
        ax2.set_title("Model Loss")
        ax2.set(xlabel='Epoch', ylabel='Loss')
        ax2.plot(history.history['loss'])
        ax2.plot(history.history['val_loss'])
        ax2.legend(['Train', 'Test'], loc='upper right')
        plt.savefig(self.exprFilePath + '/logs/training/combined.png')
        plt.close()

        for key, value in history.history.items():
            with open(self.exprFilePath + '/logs/training/' + key + ".csv", 'w') as file:
                [file.write(str(num)+",") if i != len(value) - 1 else file.write(str(num)) for i, num in enumerate(value)]


        return

    def copyFiles(self):
        for file in self.copyFileList:
            copyfile(file, self.baseFolder + "/" + str(self.exprTimeStamp) + '/files' + "/" + file.split('/')[-1])
        return

    def setupExprDir(self):

        os.makedirs(self.baseFolder + "/" + self.topic + "/" + str(self.exprTimeStamp), exist_ok=True)
        self.exprFilePath = self.baseFolder + "/" + self.topic + "/" + str(self.exprTimeStamp)
        os.makedirs(self.exprFilePath + '/weights', exist_ok=True)
        os.makedirs(self.exprFilePath + '/logs', exist_ok=True)
        os.makedirs(self.exprFilePath + '/logs/model', exist_ok=True)
        os.makedirs(self.exprFilePath + '/logs/training', exist_ok=True)
        os.makedirs(self.exprFilePath + '/files', exist_ok=True)

        self.exprWeightPath = self.exprFilePath + '/weights' + "/" + "weights-improvement-{epoch:02d}-{val_accuracy:.4f}.hdf5"
        self.copyFiles()

        if(self.currModel):
            with open(self.baseFolder + "/" + self.topic + "/" + str(self.exprTimeStamp) + '/logs/model' + '/summary.txt', 'w') as file:
                self.currModel.summary(print_fn=lambda x: file.write(x + '\n'))
        plot_model(self.currModel, to_file=self.exprFilePath + '/logs/model/model.png')
        return

    def cleanUpWeights(self):
        files = [f for f in listdir(self.exprFilePath + '/weights') if join(self.exprFilePath + '/weights', f)]
        files.pop()
        for file in files:
            if os.path.isfile(self.exprFilePath + '/weights/' + file) and os.path.splitext(file)[1] == '.hdf5':
                os.remove(self.exprFilePath + '/weights/' + file)
        return

    def setModels(self, models):
        self.models = models
        return

    def setCurrModel(self, model):
        self.currModel = model
        return

    def setArgList(self, argList):
        self.argList = argList
        return

    def setCurrArgs(self, currArgs):
        self.currArgs = currArgs
        return

    def setTopic(self, topic):
        self.topic = topic
        return

    def addCopyFile(self, file):
        self.copyFileList.append(file)
        return

    def setCopyFileList(self, files):
        self.copyFileList = files
        return

In [ ]:
# face.py
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install Keras-Applications

import os
import glob
import time
import keras
from PIL import Image
from os import listdir
from shutil import copyfile
from os.path import isfile, join
from matplotlib import pyplot as plt
from keras import applications
import keras_vggface
print(keras_vggface.__version__)
from keras_vggface.vggface import VGGFace
from keras.engine import Input
from keras.models import Model
import tensorflow as tf
import numpy as np
from keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras_vggface.utils import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
# from MLEXPS import *
from keras import backend as K
import random
from os import chdir as cd

random.seed(42)
tf.random.set_seed(42)

# Providing more training examples within certain distributions of age, gender, and race will increase the model's accuracy.

Height = 224
Width  = 224
BatchSize = 24
lr_rate=.0015
Version = 5
load_model = False
model_path = '/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/'
accuracy = 0
accuracyCount = 0
trainableCount = 30

def SaveModelImage(Model, Title):
    keras.utils.plot_model(Model, to_file=Title, show_shapes=True, show_layer_names=True)
    return

def Summary(Model):
    print(Model.summary())
    return


def MakeModel(dlsize):
    BaseModel = VGGFace(model='senet50', include_top=False, input_shape=(Height, Width, 3), pooling='avg')
    last_layer = BaseModel.get_layer('avg_pool').output

    x = keras.layers.Flatten(name='flatten')(last_layer)

    x = keras.layers.Dense(128, kernel_regularizer = keras.regularizers.l2(l = 0.015), activation='relu')(x)
    x = keras.layers.Dropout(rate=.4, seed=42)(x)

    out = keras.layers.Dense(2, activation='softmax', name='classifier')(x)
    DerivedModel = keras.Model(BaseModel.input, out)

    # # Everything in the DerivedModel is frozen
    # # Only top 30 layers are trainable
    
    # base
    for layer in DerivedModel.layers:
        layer.trainable = False
    for layer in DerivedModel.layers[-trainableCount:]:
        layer.trainable = True


    DerivedModel.compile(keras.optimizers.Adam(lr=lr_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return DerivedModel

def clearWeights(model):
    weights = model.get_weights()
    for weight in weights:
        weight = K.zeros(weight.shape, dtype=np.float64)
    model.set_weights(weights)
    return model

def preprocess_input_new(x):
    img = preprocess_input(keras.preprocessing.image.img_to_array(x), version = 2)
    return keras.preprocessing.image.array_to_img(img)

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):

  def __init__(self, trainableCount=30):
    print('working')
    super(EarlyStoppingAtMinLoss, self).__init__()
    self.epochCount = []
    self.trainableCount = trainableCount
    self.max = 0

  def on_train_begin(self, logs=None):
    self.accuracyCount = 0
    self.accuracy = 0

  def on_epoch_end(self, epoch, logs=None):
    self.max = len(self.model.layers)
    print("Ending Epoch")
    if logs['val_accuracy'] > self.accuracy:
        self.accuracy = logs['val_accuracy']
        self.accuracyCount = 0
    else:
        self.accuracyCount+=1

    if self.accuracyCount >= 10 * (len(self.epochCount)+1):
        self.epochCount.append(epoch)
        print('Adding train layers')
        self.accuracyCount = 0
        self.trainableCount += 10
        if self.trainableCount >= self.max:
            self.trainableCount = self.max
        for layer in self.model.layers:
            layer.trainable = False
        for layer in self.model.layers[-self.trainableCount:]:
            layer.trainable = True
        self.model.compile(keras.optimizers.Adam(lr=lr_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    print(self.epochCount)

if __name__ == "__main__":
    timestr = time.strftime("%Y%m%d-%H%M%S")
    model = MakeModel(1024)
    model.compile(keras.optimizers.Adam(lr=lr_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    TrainPath = '/content/drive/MyDrive/Kaggle-Autism/Autism-Data/train/'
    ValidPath = '/content/drive/MyDrive/Kaggle-Autism/Autism-Data/valid/'
    TestPath  = '/content/drive/MyDrive/Kaggle-Autism/Autism-Data/test/'

    TrainGen = keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input_new,
            horizontal_flip=True,
            rotation_range=45,
            width_shift_range=.01,
            height_shift_range=.01).flow_from_directory(
            TrainPath,
            target_size=(Height, Width),
            batch_size=BatchSize)

    ValidGen = keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input_new).flow_from_directory(
            ValidPath,
            target_size=(Height, Width),
            batch_size=BatchSize,
            shuffle=False)

    TestGen = keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input_new).flow_from_directory(
            TestPath,
            target_size=(Height, Width),
            batch_size=BatchSize,
            shuffle=False)

    os.makedirs("/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp", exist_ok=True)
    filepath = "/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/" + "weights-improvement-{epoch:02d}-{val_accuracy:.4f}.hdf5"
    SaveModelImage(model, "/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/" + "Graph.png")
    copyfile('/content/drive/MyDrive/Kaggle-Autism/face.py', "/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/face.py")
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=5, min_lr=0.00001)
    ModelCallbacks = keras.callbacks.LambdaCallback(
                            on_epoch_begin=None,
                            on_epoch_end=None,
                            on_batch_begin=None,
                            on_batch_end=None,
                            on_train_begin=None,
                            on_train_end=None)


    first = 5
    if not load_model:
        # data = model.fit_generator(
        #        generator = TrainGen,
        #        validation_data= ValidGen,
        #        epochs=first,
        #        callbacks=[ModelCallbacks, reduce_lr, checkpoint],
        #        verbose=1)
        models = [model]
        args = [{'generator':TrainGen,
                 'validation_data':TestGen,
                 'epochs':first,
                 'callbacks':[ModelCallbacks, reduce_lr, EarlyStoppingAtMinLoss()],
                 'verbose':1}]

        ml = MLEXPS()
        ml.setTopic('Autism')
        ml.setCopyFileList(['/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/face.py'])
        ml.setModels(models)
        ml.setArgList(args)
        ml.generator = True
        ml.saveBestOnly = False
        ml.startExprQ()
    else:
        model = load_model(model_path)
    
    model.save(model_path)
    Y_pred = model.predict_generator(TestGen)
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    print('Classification Report')
    target_names = ['Autistic', 'Non_Autistic']
    print(classification_report(TestGen.classes, y_pred, target_names=target_names))

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-fx4eq4x7
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-fx4eq4x7
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp37-none-any.whl size=8312 sha256=30f02a9d6e15d53501ea0db524db97471bf33e034f0b6fca32e505e3e6b29b18
  Stored in directory: /tmp/pip-ephem-wheel-cache-efglvx3_/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
0.6
104947712/104944616 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1/7

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
106/106 [==============================] - 1117s 10s/step - loss: 3.9942 - accuracy: 0.6454 - val_loss: 1.4492 - val_accuracy: 0.8033
Ending Epoch
[]

Epoch 00001: val_accuracy improved from -inf to 0.80333, saving model to /content/drive/MyDrive/Kaggle-Autism/Autism/timestamp/weights/weights-improvement-01-0.8033.hdf5
Epoch 2/5
106/106 [==============================] - 527s 5s/step - loss: 1.1632 - accuracy: 0.7778 - val_loss: 1.0974 - val_accuracy: 0.7600
Ending Epoch
[]

Epoch 00002: val_accuracy did not improve from 0.80333
Epoch 3/5
106/106 [==============================] - 527s 5s/step - loss: 0.7073 - accuracy: 0.8036 - val_loss: 0.5251 - val_accuracy: 0.8533
Ending Epoch
[]

Epoch 00003: val_accuracy improved from 0.80333 to 0.85333, saving model to /content/drive/MyDrive/Kaggle-Autism/Autism/timestamp/weights/weights-improvement-03-0.8533.hdf5
Epoch 4/5
106/106 [==============================] - 528s 5s/step - loss: 0.5145 - accuracy: 0.8392 - val_loss: 0.4326 - va

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix
Classification Report
              precision    recall  f1-score   support

    Autistic       0.89      0.83      0.86       150
Non_Autistic       0.84      0.89      0.87       150

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.86       300
weighted avg       0.86      0.86      0.86       300



In [ ]:
# Model: "model"
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #
# =================================================================
# input_1 (InputLayer)         [(None, 224, 224, 3)]     0
# _________________________________________________________________
# conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0
# _________________________________________________________________
# conv1 (Conv2D)               (None, 112, 112, 32)      864
# _________________________________________________________________
# conv1_bn (BatchNormalization (None, 112, 112, 32)      128
# _________________________________________________________________
# conv1_relu (ReLU)            (None, 112, 112, 32)      0
# _________________________________________________________________
# conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288
# _________________________________________________________________
# conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128
# _________________________________________________________________
# conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0
# _________________________________________________________________
# conv_pw_1 (Conv2D)           (None, 112, 112, 64)      2048
# _________________________________________________________________
# conv_pw_1_bn (BatchNormaliza (None, 112, 112, 64)      256
# _________________________________________________________________
# conv_pw_1_relu (ReLU)        (None, 112, 112, 64)      0
# _________________________________________________________________
# conv_pad_2 (ZeroPadding2D)   (None, 113, 113, 64)      0
# _________________________________________________________________
# conv_dw_2 (DepthwiseConv2D)  (None, 56, 56, 64)        576
# _________________________________________________________________
# conv_dw_2_bn (BatchNormaliza (None, 56, 56, 64)        256
# _________________________________________________________________
# conv_dw_2_relu (ReLU)        (None, 56, 56, 64)        0
# _________________________________________________________________
# conv_pw_2 (Conv2D)           (None, 56, 56, 128)       8192
# _________________________________________________________________
# conv_pw_2_bn (BatchNormaliza (None, 56, 56, 128)       512
# _________________________________________________________________
# conv_pw_2_relu (ReLU)        (None, 56, 56, 128)       0
# _________________________________________________________________
# conv_dw_3 (DepthwiseConv2D)  (None, 56, 56, 128)       1152
# _________________________________________________________________
# conv_dw_3_bn (BatchNormaliza (None, 56, 56, 128)       512
# _________________________________________________________________
# conv_dw_3_relu (ReLU)        (None, 56, 56, 128)       0
# _________________________________________________________________
# conv_pw_3 (Conv2D)           (None, 56, 56, 128)       16384
# _________________________________________________________________
# conv_pw_3_bn (BatchNormaliza (None, 56, 56, 128)       512
# _________________________________________________________________
# conv_pw_3_relu (ReLU)        (None, 56, 56, 128)       0
# _________________________________________________________________
# conv_pad_4 (ZeroPadding2D)   (None, 57, 57, 128)       0
# _________________________________________________________________
# conv_dw_4 (DepthwiseConv2D)  (None, 28, 28, 128)       1152
# _________________________________________________________________
# conv_dw_4_bn (BatchNormaliza (None, 28, 28, 128)       512
# _________________________________________________________________
# conv_dw_4_relu (ReLU)        (None, 28, 28, 128)       0
# _________________________________________________________________
# conv_pw_4 (Conv2D)           (None, 28, 28, 256)       32768
# _________________________________________________________________
# conv_pw_4_bn (BatchNormaliza (None, 28, 28, 256)       1024
# _________________________________________________________________
# conv_pw_4_relu (ReLU)        (None, 28, 28, 256)       0
# _________________________________________________________________
# conv_dw_5 (DepthwiseConv2D)  (None, 28, 28, 256)       2304
# _________________________________________________________________
# conv_dw_5_bn (BatchNormaliza (None, 28, 28, 256)       1024
# _________________________________________________________________
# conv_dw_5_relu (ReLU)        (None, 28, 28, 256)       0
# _________________________________________________________________
# conv_pw_5 (Conv2D)           (None, 28, 28, 256)       65536
# _________________________________________________________________
# conv_pw_5_bn (BatchNormaliza (None, 28, 28, 256)       1024
# _________________________________________________________________
# conv_pw_5_relu (ReLU)        (None, 28, 28, 256)       0
# _________________________________________________________________
# conv_pad_6 (ZeroPadding2D)   (None, 29, 29, 256)       0
# _________________________________________________________________
# conv_dw_6 (DepthwiseConv2D)  (None, 14, 14, 256)       2304
# _________________________________________________________________
# conv_dw_6_bn (BatchNormaliza (None, 14, 14, 256)       1024
# _________________________________________________________________
# conv_dw_6_relu (ReLU)        (None, 14, 14, 256)       0
# _________________________________________________________________
# conv_pw_6 (Conv2D)           (None, 14, 14, 512)       131072
# _________________________________________________________________
# conv_pw_6_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_pw_6_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_dw_7 (DepthwiseConv2D)  (None, 14, 14, 512)       4608
# _________________________________________________________________
# conv_dw_7_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_dw_7_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_pw_7 (Conv2D)           (None, 14, 14, 512)       262144
# _________________________________________________________________
# conv_pw_7_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_pw_7_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_dw_8 (DepthwiseConv2D)  (None, 14, 14, 512)       4608
# _________________________________________________________________
# conv_dw_8_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_dw_8_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_pw_8 (Conv2D)           (None, 14, 14, 512)       262144
# _________________________________________________________________
# conv_pw_8_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_pw_8_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)       4608
# _________________________________________________________________
# conv_dw_9_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_dw_9_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_pw_9 (Conv2D)           (None, 14, 14, 512)       262144
# _________________________________________________________________
# conv_pw_9_bn (BatchNormaliza (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_pw_9_relu (ReLU)        (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_dw_10 (DepthwiseConv2D) (None, 14, 14, 512)       4608
# _________________________________________________________________
# conv_dw_10_bn (BatchNormaliz (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_dw_10_relu (ReLU)       (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_pw_10 (Conv2D)          (None, 14, 14, 512)       262144
# _________________________________________________________________
# conv_pw_10_bn (BatchNormaliz (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_pw_10_relu (ReLU)       (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_dw_11 (DepthwiseConv2D) (None, 14, 14, 512)       4608
# _________________________________________________________________
# conv_dw_11_bn (BatchNormaliz (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_dw_11_relu (ReLU)       (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_pw_11 (Conv2D)          (None, 14, 14, 512)       262144
# _________________________________________________________________
# conv_pw_11_bn (BatchNormaliz (None, 14, 14, 512)       2048
# _________________________________________________________________
# conv_pw_11_relu (ReLU)       (None, 14, 14, 512)       0
# _________________________________________________________________
# conv_pad_12 (ZeroPadding2D)  (None, 15, 15, 512)       0
# _________________________________________________________________
# conv_dw_12 (DepthwiseConv2D) (None, 7, 7, 512)         4608
# _________________________________________________________________
# conv_dw_12_bn (BatchNormaliz (None, 7, 7, 512)         2048
# _________________________________________________________________
# conv_dw_12_relu (ReLU)       (None, 7, 7, 512)         0
# _________________________________________________________________
# conv_pw_12 (Conv2D)          (None, 7, 7, 1024)        524288
# _________________________________________________________________
# conv_pw_12_bn (BatchNormaliz (None, 7, 7, 1024)        4096
# _________________________________________________________________
# conv_pw_12_relu (ReLU)       (None, 7, 7, 1024)        0
# _________________________________________________________________
# conv_dw_13 (DepthwiseConv2D) (None, 7, 7, 1024)        9216
# _________________________________________________________________
# conv_dw_13_bn (BatchNormaliz (None, 7, 7, 1024)        4096
# _________________________________________________________________
# conv_dw_13_relu (ReLU)       (None, 7, 7, 1024)        0
# _________________________________________________________________
# conv_pw_13 (Conv2D)          (None, 7, 7, 1024)        1048576
# _________________________________________________________________
# conv_pw_13_bn (BatchNormaliz (None, 7, 7, 1024)        4096
# _________________________________________________________________
# conv_pw_13_relu (ReLU)       (None, 7, 7, 1024)        0
# _________________________________________________________________
# global_average_pooling2d (Gl (None, 1024)              0
# _________________________________________________________________
# dense (Dense)                (None, 128)               131200
# _________________________________________________________________
# dropout (Dropout)            (None, 128)               0
# _________________________________________________________________
# dense_1 (Dense)              (None, 2)                 258
# =================================================================
# Total params: 3,360,322
# Trainable params: 3,338,434
# Non-trainable params: 21,888
# _________________________________________________________________
# None

# Fully       =  3
# conv_dw_13  = 10
# conv_pad_12 = 17
# conv_dw_11  = 23
# conv_dw_10  = 29
# conv_dw_9   = 35
# conv_dw_8   = 41
# conv_dw_7   = 47
# conv_pad_6  = 54
# conv_dw_5   = 60
# conv_pad_4  = 67
# conv_dw_3   = 73
# conv_pad_2  = 80
# conv_dw_1   = 86
# conv1_pad   = 90
# input       = 91